In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
import time
import random
from tqdm import tqdm
import pathlib


#Torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
import torchvision.transforms as T

#sklearn
from sklearn.model_selection import StratifiedKFold
from skimage import io

################# DETR FUCNTIONS FOR LOSS #######################
import sys
sys.path.append('./detr_custom/')

from models.matcher import HungarianMatcher
from models.detr import SetCriterion
import engine
#################################################################

import matplotlib.pyplot as plt

#Glob
from glob import glob

from typing import Iterable, Sequence, List, Tuple, Dict, Optional, Union, Any, Callable, Mapping
from types import ModuleType
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
from generators import BlenderStandardDataset, TorchStandardDataset, TorchStereoDataset
import importlib
from pprint import pprint
import sqlite3 as db
from functools import wraps

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
    
def pytorch_init_janus_gpu():
    device_id = 1
    torch.cuda.set_device(device_id)
    
    # Sanity checks
    assert torch.cuda.current_device() == 1, 'Using wrong GPU'
    assert torch.cuda.device_count() == 2, 'Cannot find both GPUs'
    assert torch.cuda.get_device_name(0) == 'GeForce RTX 2080 Ti', 'Unexpected GPU name'
    assert torch.cuda.is_available() == True, 'GPU not available'
    return torch.device('cuda', device_id)


def reloader(module_or_member: Union[ModuleType, Any]):    
    if isinstance(module_or_member, ModuleType):
        importlib.reload(module_or_member)
        return module
    else:
        module = importlib.import_module(module_or_member.__module__)
        importlib.reload(module)
        return module.__dict__[module_or_member.__name__]

In [3]:
print(torch.cuda.get_device_name('cuda:1'))
print('Memory Usage:')
print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [4]:
seed = 42069
seed_everything(seed)

try:
    device = pytorch_init_janus_gpu()
    print(f'Using device: {device} ({torch.cuda.get_device_name()})')
except AssertionError as e:
    print('GPU could not initialize, got error:', e)
    device = torch.device('cpu')
    print('Device is set to CPU')

Using device: cuda:1 (GeForce RTX 2080 Ti)


<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

In [5]:
TORCH_CACHE_DIR = 'torch_cache'
DATASET_DIR = '/mnt/blendervol/objdet_std_data'
DATASET_DIR = '/mnt/blendervol/leftright_left_data'
SQL_TABLE = 'bboxes_std'
WEIGHTS_DIR = 'fish_statedicts'
NUM_CLASSES=6+1

In [6]:
db_con = db.connect(f'file:{os.path.join(DATASET_DIR,"bboxes.db")}?mode=ro', uri=True)

In [7]:
num2name = eval(open(os.path.join(DATASET_DIR,"metadata.txt"), 'r').read())

In [8]:
n_data = pd.read_sql_query('SELECT COUNT(DISTINCT(imgnr)) FROM bboxes_std', db_con).values[0][0]

TRAIN_RANGE = (0, int(3/4*n_data))
VAL_RANGE = (int(3/4*n_data), n_data)

In [9]:
torch.hub.set_dir(TORCH_CACHE_DIR)

<!--  
class DETRModel(nn.Module):
    def __init__(self, num_classes, num_queries):
        super(DETRModel,self).__init__()
        self.num_classes = num_classes
        self.num_queries = num_queries
        
        self.model = torch.hub.load(
            os.path.join(TORCH_CACHE_DIR, 'facebookresearch_detr_master'), 
            model='detr_resnet50', 
            pretrained=True,
            source='local'
        )
        
        self.in_features = self.model.class_embed.in_features
        
        self.model.class_embed = nn.Linear(in_features=self.in_features,out_features=self.num_classes)
        self.model.num_queries = self.num_queries
        
    def forward(self,images):
        return self.model(images)

model = DETRModel(NUM_CLASSES, 100)
model = model.to(device)
-->

In [141]:
def save_model(obj: Any, f: str):
    pathlib.Path(f).parent.mkdir(parents=True, exist_ok=True)
    assert isinstance(f, str), "Filename must be of type string when saving model"
    torch.save(obj=obj, f=f)
    
    
def interruptable(f: Callable):
    '''Decorator for functions that should handle KeyboardInterrupts more gracefully'''
    @wraps(f)
    def wrapper(*args, **kwargs):
        try:
            retval = f(*args, **kwargs)
        except KeyboardInterrupt as e:
            print('Process interrupted')
            return 
        return retval
    return wrapper


def img_handler(images: List[Tuple[torch.Tensor, torch.Tensor]]):
    # pp for pre process
    pp = T.Compose([
        T.Resize(800),
        T.Normalize([0.485, 0.456, 0.406], 
                    [0.229, 0.224, 0.225])
    ])
    
    return [
        (pp(pair[0]).to(device), pp(pair[1]).to(device)) 
        for pair in images
    ]
    
    
def label_handler(labels):
    return [{k: v.to(device) for k, v in t.items()} for t in labels]

    
def _validate_model(context, traintqdminfo):
    running_val_loss = 0.0

    # valbar will disappear after it is done since leave=False
    valbar = tqdm(
        iterable=enumerate(context['valloader'], 0), 
        total=len(context['valloader']), 
        unit=' batches',
        desc=f' Validating',
        ascii=True,
        position=0,
        leave=False
    )
    
    model = context['model']
    criterion = context['criterion']
    
    model.eval()
    criterion.eval()

    # Loop through val batches
    with torch.no_grad():
        for i, (images, labels) in valbar:
            # images = [image.to(device) for image in images]
            images = img_handler(images)
            labels = label_handler(labels)

            output = model(images)

            loss_dict = criterion(output, labels)
            weight_dict = criterion.weight_dict
            losses = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)

            # print statistics
            running_val_loss += losses.item()
            val_loss = running_val_loss / (i+1)
            valtqdminfo = {**traintqdminfo, 'val loss':val_loss}
            valbar.set_postfix(valtqdminfo)
            
    return valtqdminfo


def _train_model(context, epoch, n_epochs, leave_tqdm):
    running_train_loss = 0.0

    trainbar = tqdm(
        iterable=enumerate(context['trainloader'], 0),
        total=len(context['trainloader']),
        unit=' batches',
        desc=f' Epoch {epoch+1}/{n_epochs}',
        ascii=True,
        position=0,
        leave=leave_tqdm
    )

    model.train()
    criterion.train()

    # Loop through train batches
    for i, (images, labels) in trainbar:
        # images = [image.to(device) for image in images]
        images = img_handler(images)
        labels = label_handler(labels)

        output = model(images[0])
        loss_dict = criterion(output, labels)
        weight_dict = criterion.weight_dict
        losses = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)

        # Zero parameter gradients since PyTorch will just accumulate the gradient
        # vectors while it trains (in order to get the "mean" direction to move in
        # the parameter space). Also doing it this way minimizes memory allocation
        # etc, probably.
        optimizer.zero_grad()

        losses.backward() # Computes gradients
        optimizer.step() # Do a gradient step

        # print statistics
        running_train_loss += losses.item()
        train_loss = running_train_loss / (i+1)
        traintqdminfo = {'train loss':train_loss}
        trainbar.set_postfix(traintqdminfo)
    
    return trainbar, traintqdminfo


@interruptable
def train_model(
        trainloader: DataLoader, 
        valloader: DataLoader, 
        model: nn.Module, 
        criterion, 
        optimizer, 
        n_epochs: int, 
        device: torch.device, 
        validate: bool = True,
        save_best: bool = True
    ):
    
    # for convenience
    context = {
        'trainloader':trainloader,
        'valloader':valloader,
        'model':model,
        'criterion':criterion,
        'optimizer':optimizer,
        'device':device
    }
    
    best_val_loss = np.inf
    
    for epoch in range(n_epochs):
        trainbar, traintqdminfo = _train_model(context, epoch, n_epochs, not validate)
            
        if validate:
            valtqdminfo = _validate_model(context, traintqdminfo)
        
            # Extra dirty tqdm hack hehe
            # _validate_model will create its own tqdm bar that will replace the bar
            # from _train_model, but will clear itself afterwards
            # the code below reactivates the previous train bar
            trainbar.disable = False
            trainbar.set_postfix({**traintqdminfo, **valtqdminfo})
            trainbar.disable = True
            print(file=sys.stderr)
        
            # Save best models
            if save_best:
                if valtqdminfo['val loss'] < best_val_loss:
                    best_val_loss = valtqdminfo['val loss']
                    isodatenow = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
                    daydir = datetime.today().strftime("weights_%Y-%m-%d")
                    filename = (
                        f'detr_statedicts_epoch{epoch+1}'
                        f'_train{traintqdminfo["train loss"]:.4f}_val{best_val_loss:.4f}'
                        f'_{isodatenow}.pth'
                    )
                    filepath = os.path.join(WEIGHTS_DIR, daydir, filename)

                    save_model(
                        obj={
                            'model':model.state_dict(),
                            'optimizer':optimizer.state_dict(),
                            'criterion':criterion.state_dict(),
                        },
                        f = filepath
                    )
        
            

<!--
from torchvision.models import resnet50
class DETRdemo(nn.Module):
    """
    Demo DETR implementation.

    Demo implementation of DETR in minimal number of lines, with the
    following differences wrt DETR in the paper:
    * learned positional encoding (instead of sine)
    * positional encoding is passed at input (instead of attention)
    * fc bbox predictor (instead of MLP)
    The model achieves ~40 AP on COCO val5k and runs at ~28 FPS on Tesla V100.
    Only batch size 1 supported.
    """
    def __init__(self, num_classes, hidden_dim=256, nheads=8,
                 num_encoder_layers=6, num_decoder_layers=6):
        super().__init__()

        # create ResNet-50 backbone
        self.backbone = resnet50()
        del self.backbone.fc

        # create conversion layer
        self.conv = nn.Conv2d(2048, hidden_dim, 1)

        
        # prediction heads, one extra class for predicting non-empty slots
        # note that in baseline DETR linear_bbox layer is 3-layer MLP
        self.linear_class = nn.Linear(hidden_dim, num_classes + 1)
        self.linear_bbox = nn.Linear(hidden_dim, 4)
        
        # create a default PyTorch transformer
        self.transformer = nn.Transformer(
            hidden_dim, nheads, num_encoder_layers, num_decoder_layers)

        # output positional encodings (object queries)
        self.query_pos = nn.Parameter(torch.rand(100, hidden_dim))

        # spatial positional encodings
        # note that in baseline DETR we use sine positional encodings
        self.row_embed = nn.Parameter(torch.rand(50, hidden_dim // 2))
        self.col_embed = nn.Parameter(torch.rand(50, hidden_dim // 2))

    def forward(self, inputs):
        # propagate inputs through ResNet-50 up to avg-pool layer
        if not isinstance(inputs, torch.Tensor):
            inputs = torch.stack(inputs, dim=0).to(device)

        x = self.backbone.conv1(inputs)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        # convert from 2048 to 256 feature planes for the transformer
        h = self.conv(x)

        # construct positional encodings
        H, W = h.shape[-2:]
        pos = torch.cat([
            self.col_embed[:W].unsqueeze(0).repeat(H, 1, 1),
            self.row_embed[:H].unsqueeze(1).repeat(1, W, 1),
        ], dim=-1).flatten(0, 1).unsqueeze(1)

    
        # propagate through the transformer
        h = self.transformer(pos + 0.1 * h.flatten(2).permute(2, 0, 1),
                             self.query_pos.unsqueeze(1)).transpose(0, 1)
        
        print(h.shape)
    
        # finally project transformer outputs to class labels and bounding boxes
        return {'pred_logits': self.linear_class(h), 
                'pred_boxes': self.linear_bbox(h).sigmoid()}

def prepare_model(model, num_classes):
    state_dict = torch.hub.load_state_dict_from_url(
        url='https://dl.fbaipublicfiles.com/detr/detr_demo-da2a99e9.pth',
        map_location='cpu',
        check_hash=True
    )
    
    state_dict = torch.load('fish_statedicts/weights_2021-01-06/detr_statedicts_epoch164_train0.2128_val0.3822_2021-01-06T00:58:28.pth')
    model.linear_class = nn.Linear(model.linear_bbox.in_features, num_classes + 1)
    model.load_state_dict(state_dict['model'])
    return model
    
model = DETRdemo(91)
model = prepare_model(model, 6)
model = model.to(device)
-->

In [142]:
from torchvision.models import resnet50

def dict_union_update(a: dict, b: dict):
    '''Updates "a" with the union of "a" and "b"'''
    a.update((                                   # Set union
        (key, b.get(key, a.get(key))) for key in a.keys() & b.keys()
    ))
    

class Encoder(nn.Module):
    def __init__(self, hidden_dim=256, nheads=8,
                 num_encoder_layers=6, num_decoder_layers=6, pretrained: bool=True):
        super().__init__()

        # create ResNet-50 backbone
        self.backbone = resnet50()
        del self.backbone.fc

        # create conversion layer
        self.conv = nn.Conv2d(2048, hidden_dim, 1)
        
        # create a default PyTorch transformer
        self.transformer = nn.Transformer(
            hidden_dim, nheads, num_encoder_layers, num_decoder_layers, dropout=0)

        # output positional encodings (object queries)
        self.query_pos = nn.Parameter(torch.rand(100, hidden_dim))

        # spatial positional encodings
        # note that in baseline DETR we use sine positional encodings
        self.row_embed = nn.Parameter(torch.rand(50, hidden_dim // 2))
        self.col_embed = nn.Parameter(torch.rand(50, hidden_dim // 2))
        
        if pretrained:
            self.load_pretrained_weights()
    
    def load_pretrained_weights(self):
        state_dict = torch.hub.load_state_dict_from_url(
            url='https://dl.fbaipublicfiles.com/detr/detr_demo-da2a99e9.pth',
            map_location='cpu',
            check_hash=True
        )
    
        self_state_dict = self.state_dict()
        dict_union_update(self_state_dict, state_dict)
        self.load_state_dict(self_state_dict)
        print('Encoder successfully loaded with pretrained weights')

    def forward(self, inputs):
        # propagate inputs through ResNet-50 up to avg-pool layer
        if not isinstance(inputs, torch.Tensor):
            inputs = torch.stack(inputs, dim=0).to(device)

        x = self.backbone.conv1(inputs)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        # convert from 2048 to 256 feature planes for the transformer
        h = self.conv(x)

        # construct positional encodings
        H, W = h.shape[-2:]
        pos = torch.cat([
            self.col_embed[:W].unsqueeze(0).repeat(H, 1, 1),
            self.row_embed[:H].unsqueeze(1).repeat(1, W, 1),
        ], dim=-1).flatten(0, 1).unsqueeze(1)

    
        # propagate through the transformer
        h = self.transformer(pos + 0.1 * h.flatten(2).permute(2, 0, 1),
                             self.query_pos.unsqueeze(1)).transpose(0, 1)
        
        return h
    
    
class Decoder(nn.Module):
    def __init__(self, num_classes, hidden_dim: int=256):
        '''
        num_classes: int, should be number of classes WITHOUT "no object" class
        '''
        super().__init__()
        
        merge_hidden_dim = 64
                
        # For latent space merging, use 1x1 convs
        self.merger1 = nn.Conv2d(in_channels=2, out_channels=merge_hidden_dim, kernel_size=1, stride=1)
        self.merger2 = nn.Conv2d(in_channels=merge_hidden_dim, out_channels=merge_hidden_dim, kernel_size=1, stride=1)
        self.merger3 = nn.Conv2d(in_channels=merge_hidden_dim, out_channels=1, kernel_size=1, stride=1)
        
        self.linear_pre_class = nn.Linear(in_features=hidden_dim, out_features=hidden_dim)
        self.linear_pre_bbox = nn.Linear(in_features=hidden_dim, out_features=hidden_dim)
        
        # prediction heads, one extra class for predicting non-empty slots
        # note that in baseline DETR linear_bbox layer is 3-layer MLP
        self.linear_class = nn.Linear(hidden_dim, num_classes + 1)
        self.linear_bbox = nn.Linear(hidden_dim, 4)
    
    
    def merge(self, h_left, h_right):
        # (2, 100, 256)
        h1 = torch.cat((h_left, h_right), dim=0).unsqueeze(0) # 2 channel out
        
        h1 = self.merger1(h1) # 64 channel out
        h1 = F.relu(h1)
        h2 = self.merger2(h1) # 64 channel out
        h2 = F.relu(h1+h2)  # Skip connection
        h2 = self.merger3(h2) # 1 channel out
        h2 = F.relu(h2)
        
        # (1, 100, 256)
        return h2
        
    def forward(self, h_left, h_right):
        h = self.merge(h_left, h_right)[0]
        
        h_logits = F.relu(self.linear_pre_class(h))
        h_boxes = F.relu(self.linear_pre_bbox(h))
        
        # finally project transformer outputs to class labels and bounding boxes
        return {'pred_logits': self.linear_class(h_logits),
                'pred_boxes': self.linear_bbox(h_boxes).sigmoid()}
    

class FishDETR(nn.Module):
    """
    Demo DETR implementation.

    Demo implementation of DETR in minimal number of lines, with the
    following differences wrt DETR in the paper:
    * learned positional encoding (instead of sine)
    * positional encoding is passed at input (instead of attention)
    * fc bbox predictor (instead of MLP)
    The model achieves ~40 AP on COCO val5k and runs at ~28 FPS on Tesla V100.
    Only batch size 1 supported.
    """
    def __init__(self, hidden_dim: int=256, freeze_encoder: bool=True):
        '''
        num_classes: int, should be number of classes WITHOUT "no object" class
        '''
        super().__init__()

        self.encoder = Encoder(hidden_dim=hidden_dim)
        self.decoder = Decoder(6)
        
        if freeze_encoder:
            self.freeze_module(self.encoder)
            print('Encoder layers are frozen')
            
        self.freeze_encoder = freeze_encoder
        
    @staticmethod
    def freeze_module(module):
        for param in module.parameters():
                param.requires_grad = False
        
    def forward(self, imgs: Tuple[torch.Tensor, torch.Tensor]):
        # (1, 100, 256)
        # (batchsize, n_queries, embedding_dim)
        # imgs[0] and imgs[1] should be (N, C, H, W)
        if self.freeze_encoder:
            self.encoder.eval()
        
        h_left = self.encoder(imgs[0])
        h_right = self.encoder(imgs[1])
        return self.decoder(h_left, h_right)
    
model = FishDETR()
model = model.to(device)

Encoder successfully loaded with pretrained weights
Encoder layers are frozen


In [169]:
h_lefts = []
h_rights = []
ys = []
for x, y in traingen:
    x = img_handler((x,))[0]
    h_lefts.append(model.encoder(x[0]))
    h_rights.append(model.encoder(x[1]))
    ys.append(y)

class Gen:
    def __init__(self):
        self.gen = zip(h_lefts, h_rights, ys)
    
    def __len__(self):
        return len(ys)
    
    def __getitem__(self, i):
        l, r, y = next(self.gen)
        return (l, r), y
    
model.decoder.train()
gener = Gen()
for imgs, y in gener:
    output = model.decoder(imgs[0], imgs[1])
    

{'pred_logits': tensor([[[-0.0055,  0.0317, -0.1104,  0.0643,  0.0784,  0.0772,  0.0270],
         [-0.0745, -0.0170, -0.0495,  0.0999,  0.0339,  0.0738,  0.0606],
         [-0.0047,  0.0154, -0.0515,  0.0783,  0.0229,  0.0454,  0.0361],
         [-0.0404,  0.0254, -0.1163,  0.0439,  0.0073,  0.1088, -0.0013],
         [-0.0848,  0.0111, -0.0510,  0.1010,  0.0371,  0.0487,  0.0702],
         [-0.0454,  0.0451, -0.0562,  0.0463,  0.0297,  0.0905,  0.0269],
         [-0.0906,  0.0159, -0.0253,  0.0988,  0.0252,  0.0689,  0.0359],
         [-0.0767, -0.0294, -0.0588,  0.1293,  0.0773,  0.0521,  0.0512],
         [-0.0451,  0.0055, -0.0693,  0.0605, -0.0047,  0.0734,  0.0346],
         [-0.0168, -0.0376, -0.0454,  0.1020,  0.0459,  0.0477,  0.0587],
         [-0.0547, -0.0146, -0.0568,  0.1407,  0.0705,  0.0509,  0.0544],
         [-0.1000, -0.0161, -0.0646,  0.1128,  0.0853,  0.0430,  0.0562],
         [-0.0443,  0.0444, -0.1228,  0.0410,  0.0004,  0.0966, -0.0014],
         [-0.0031, -0.

In [157]:
weight_dict = {'loss_ce': 1, 'loss_bbox': 1 , 'loss_giou': 1}
losses = ['labels', 'boxes', 'cardinality']
matcher = HungarianMatcher()
criterion = SetCriterion(6, matcher, weight_dict, eos_coef = 0.5, losses=losses)
criterion = criterion.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
# optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
# optimizer = torch.optim.SGD(model.parameters(), lr=8e-5, momentum=0.001, dampening=0.000001)

BATCH_SIZE = 1
traingen = TorchStereoDataset(DATASET_DIR, SQL_TABLE, BATCH_SIZE, shuffle=False, imgnrs=range(8,12))
valgen = TorchStereoDataset(DATASET_DIR, SQL_TABLE, BATCH_SIZE, shuffle=False, imgnrs=range(1024,1024+8))

# trainloader = DataLoader(
#     dataset=traingen,
#     batch_size=1,
#     collate_fn=lambda x: tuple(zip(*x)),
# )

# valloader = DataLoader(
#     dataset=valgen,
#     batch_size=1,
#     collate_fn=lambda x: tuple(zip(*x)),
# )

trainloader = DataLoader(
    dataset=Gener(),
    batch_size=1,
    collate_fn=lambda x: tuple(zip(*x)),
)

valloader = DataLoader(
    dataset=valgen,
    batch_size=1,
    collate_fn=lambda x: tuple(zip(*x)),
)


<!--
def sanity_dataset(gen: DataLoader, index=0, classmap: Optional[Mapping[int, str]]=None):
    geniter = iter(gen)
    
    for i in range(index+1):
        img, targets = next(geniter)
    
    img = img.numpy().transpose((1,2,0))
    boxes = targets['boxes'].numpy()
    labels = targets['labels'].numpy()
    
    h, w = img.shape[:2]
    
    fig, ax = plt.subplots(1,1,figsize=(10,7))
    ax.imshow(img)
    
    for cls, box in zip(labels,boxes):
        cxy = box[:2]
        # Visualize center xy
        ax.add_patch(plt.Circle(cxy*(h, w), 5, facecolor='red', edgecolor='k', alpha=0.4))
        
        bw, bh = box[2], box[3]
        # tlxy = top left xy
        tlxy = (cxy - (bw/2, bh/2))*(h, w)
        ax.add_patch(plt.Rectangle(tlxy, bw*w, bh*h, fill=False, lw=2, color='red', alpha=0.4))
        
        try:
            ax.text(*tlxy, classmap[int(cls)], fontsize=11, bbox=dict(facecolor='red', alpha=0.5))
        except:
            pass
    

sanity_dataset(traingen, 16, num2name)

-->

In [158]:
train_model(
    trainloader,
    valloader,
    model,
    criterion,
    optimizer,
    n_epochs=20000,
    device=device,
    save_best=False,
    validate=False
)

 Epoch 1/20000:   0%|          | 0/4 [00:00<?, ? batches/s]


ValueError: too many values to unpack (expected 2)

In [ ]:
# save_model(model.state_dict(), 'overfit_range_8_12_2in1out.pth')

In [ ]:
def box_cxcywh_to_xyxy(x: torch.Tensor):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)


def plot_results(img: Image.Image, classes: Iterable, boxes: Iterable, classmap: Optional[Mapping[int, str]]=None, ax: Optional=None):
    
    if ax is None:
        fig, ax = plt.subplots(figsize=(16,10))
    img = np.array(img)
    ax.imshow(img)
    
    if len(boxes) != 0:
        h, w = img.shape[:2]
        boxes = box_cxcywh_to_xyxy(boxes)
        boxes[:,[0,2]] *= w
        boxes[:,[1,3]] *= h
        
        print([int(cls) for cls in classes])
        
        for cls, (xmin, ymin, xmax, ymax) in zip(classes, boxes):
            ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                       fill=False, color='cyan', linewidth=3))
            try:
                ax.text(xmin, ymin, classmap[int(cls)], fontsize=11, bbox=dict(facecolor='cyan', alpha=0.9))
            except:
                ax.text(xmin, ymin, str(int(cls)), fontsize=11, bbox=dict(facecolor='cyan', alpha=0.9))
                pass
    
    if ax is None:
        ax.axis('off')
        plt.show()
    return ax

    
def postprocess(logits: torch.Tensor, boxes: torch.Tensor):
#     return logits.argmax(-1), boxes
    keepmask = logits.softmax(-1)[:,:-1].max(-1)[0] > 0.2
    if any(keepmask) == False:
        return torch.Tensor(), torch.Tensor()
    return logits[keepmask].argmax(-1), boxes[keepmask]

    
def eval_model(model, img: torch.Tensor, classmap: Optional[Mapping[int, str]]=None, ax: Optional=None):    
    with torch.no_grad():
        model.eval()
        
#         output = model(img.to(device).unsqueeze(0))
        output = model((img[0].to(device), img[1].to(device)))
        
        boxes = output['pred_boxes'][0]
        logits = output['pred_logits'][0]
        print(logits.argmax(-1))
        
        logits_, boxes_ = postprocess(logits, boxes)
        
        plot_results(img[0][0].cpu().numpy().transpose((1,2,0)), logits_, boxes_, classmap, ax=ax)
        
# x, y = traingen[16]
# eval_model(model, x, num2name)

In [ ]:
def eval_model2(model: nn.Module, img: Image.Image): 
    transform = T.Compose([
        T.Resize(800),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])

    img_torch = transform(img).to(device)
    
    model.eval()
    with torch.no_grad():    
        output = model(img_torch.unsqueeze(0))
        
    boxes = output['pred_boxes'][0]
    logits = output['pred_logits'][0]
    logits_, boxes_ = postprocess(logits, boxes)
    
    
    plot_results(np.array(img), logits_, boxes_)
    
def eval_compare_model(model: nn.Module, gen: Iterable, index: int=0, classmap: Optional[Mapping[int, str]]=None):
    x, y = gen[index]
    
    fig, axes = plt.subplots(1,2,figsize=(15,7))
    eval_model(model, x, classmap, axes[0])
    plot_results(x[0][0].cpu().numpy().transpose((1,2,0)), y['labels'], y['boxes'], classmap, axes[1])
    
eval_compare_model(model, traingen, index=2, classmap=num2name)

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

In [ ]:
raise ValueError

In [ ]:
# test_img = io.imread('test_image2.png').copy() / 255
test_img = Image.open('test_image1.jpg').convert('RGB')

transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

test_img_torch = transform(test_img)
test_img_torch